# Variational AutoEncoder Generator Tensorflow
This should open a pre-trained VAE and become a Data generator

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://www.youtube.com/watch?v=bbOFvxbMIV0
* https://www.youtube.com/watch?v=P78QYjWh5sM
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pylab import *
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
latent_size = 20
SAVE_FOLDER='/tmp/vae_cnn/'

In [2]:
from models import VAE_CNN_GEN
model = VAE_CNN_GEN(latent_size = latent_size)
model_in = model.input
model_out = model.output

In [3]:
# Select only the convs
list_params = [v for v in tf.global_variables() if "z_matrix" in v.name or "conv" in v.name ]

In [4]:
list_params

[<tf.Variable 'z_matrix/kernel:0' shape=(20, 1568) dtype=float32_ref>,
 <tf.Variable 'z_matrix/bias:0' shape=(1568,) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose/kernel:0' shape=(5, 5, 16, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose_1/kernel:0' shape=(5, 5, 1, 16) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose_1/bias:0' shape=(1,) dtype=float32_ref>]

### Define saver objects to load the VAE generator variables

In [5]:
# Define the saver object to load only the conv variables
saver_load_autoencoder = tf.train.Saver(var_list=list_params)

### Build Graph and create session

In [6]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

# Restore only the weights (From AutoEncoder)
saver_load_autoencoder.restore(sess, SAVE_FOLDER+"model.ckpt-32")

INFO:tensorflow:Restoring parameters from /tmp/vae_cnn/model.ckpt-32


### Generate some input from the latent space

In [7]:
# Create a latent as a normal distribution with size 20, mean 0, variance 1
latent = np.random.normal(0, 1, [1,latent_size])

out_img = sess.run(model_out, feed_dict={model_in:latent})
plt.imshow(out_img.reshape([28,28]))
plt.show()

### Generate a video

In [8]:
dpi = 50

def ani_frame():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    im = ax.imshow(np.random.rand(28,28),cmap='gray',interpolation='nearest')
    #latent = np.random.normal(0, 1, [1,20])
    #out_img = sess.run(model_out, feed_dict={model_in:latent})
    #im = ax.imshow(out_img.reshape([28,28]),cmap='gray',interpolation='nearest')
    
    im.set_clim([0,1])
    fig.set_size_inches([5,5])


    tight_layout()


    def update_img(n):
        #tmp = rand(300,300)
        #im.set_data(tmp)
        latent = np.random.normal(0, 1, [1,latent_size])
        out_img = sess.run(model_out, feed_dict={model_in:latent})
        im.set_data(out_img.reshape([28,28]))
        return im

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,300,interval=30)
    writer = animation.writers['ffmpeg'](fps=30)

    ani.save('demo.mp4',writer=writer,dpi=dpi)
    return ani

In [9]:
ani_frame()